In [ ]:
import format
from finetuning import write_classifier_format, pull_kaggle_test_set, inference
import fetch_data


In [ ]:
# The competition:
# In recent years, large language models (LLMs) have become increasingly sophisticated, capable of generating text
# that is difficult to distinguish from human-written text.
# In this competition, we hope to foster open research and transparency on AI detection techniques applicable in the real world.
#This competition challenges participants to develop a machine learning model that can accurately detect whether an essay was written
# by a student or an LLM. The competition dataset comprises a mix of student-written essays and essays generated by a variety of LLMs.

In [ ]:
#### Datamix creation
# We mixed several datasets of LLM generated and Student generated essays.

In [ ]:
### Training
# Train dataset is created via sampling function that picks out samples of different datasources.
# Our current datasources: Persuade, daigt, fpe, OUTFOX
# Future datasources: In-house LLM generated essays

In [ ]:
# Script arguments
baseline_model = "distilbert-base-uncased"
version = "v17-01-2025"
sample_size = 1000
sources = ['fpe', 'persuade']

In [ ]:
# Load finetuned model (already finetuned on TAU cluster)
ft_model_path = f"./models/modelname_{baseline_model}_version_{version}_size_{sample_size}_sources_{'-'.join(sources)}"
baseline_model_path = f"./models/{baseline_model}"

In [ ]:
# Load test set - currently, test set is train dataset provided in Kaggle challange
test_output_path = f"./data/test_data_version_{version}_size_{sample_size}_sources_{'-'.join(sources)}"

# First, dataset is generated in dataframe format
test_set_df_format = pull_kaggle_test_set()
test_set_df_format.head()


,essay_text,generated,prompt_text
0,Cars. Cars have been around since they became ...,0,Write an explanatory essay to inform fellow ci...
1,Transportation is a large necessity in most co...,0,Write an explanatory essay to inform fellow ci...
2,"""America's love affair with it's vehicles seem...",0,Write an explanatory essay to inform fellow ci...
3,How often do you ride in a car? Do you drive a...,0,Write an explanatory essay to inform fellow ci...
4,Cars are a wonderful thing. They are perhaps o...,0,Write an explanatory essay to inform fellow ci...


In [ ]:
# Then, it is formatted to classifier format (input + label)
test_set = write_classifier_format(pull_kaggle_test_set(), output_path=test_output_path)
test_set.head()

,input,label
0,Prompt Text: Write an explanatory essay to inf...,0
1,Prompt Text: Write an explanatory essay to inf...,0
2,Prompt Text: Write an explanatory essay to inf...,0
3,Prompt Text: Write an explanatory essay to inf...,0
4,Prompt Text: Write an explanatory essay to inf...,0


In [ ]:
# Perform inference on test set in classifier format.
average_loss = inference(test_set, ft_model_path)
print(f"Average loss on test set for finetuned model is {average_loss}.")

Map:   0%|          | 0/1378 [00:00<?, ? examples/s]

Average loss on test set for finetuned model is 0.017628765122244185.


In [ ]:
# What is the loss for the baseline, distilbert model, before our finetuning?
average_loss_baseline = inference(test_set, baseline_model_path)
print(f"Average loss on test set for baseline model is {average_loss_baseline}")

Map:   0%|          | 0/1378 [00:00<?, ? examples/s]

Average loss on test set for finetuned model is 0.0169101326468033


In [ ]:
## As you can see, our initial finetuning is not very good. What's next?
# Datamix creation: while we have a wide range of data, we need to make it more robust to accommodate adversarial examples in the test set.
# In general, most contendors in the
# To do so, we found large datasets of LLM generated essays, but without prompts.
# We wrote logic that given an essay, generates a prompt corresponding to that essay via LLM (Google's flan-t5-large)

In [ ]:
health_org_essay = "i think that donal trump decision to quit the health organization is not good because people are sick and\
they cant do anything about it. i think that all the countries need to be in the health organization to cure the cancer and other disease that is\
very important for mankind"
vacation_essay = "The perfect vacation is not a one-size-fits-all concept;\
                  it is deeply personal and shaped by individual desires and priorities.\
                  For some, it might involve basking under the sun on a quiet tropical beach, where time seems to slow,\
                  and the only sound is the gentle lapping of waves. For others, it could mean exploring bustling cities filled with art,\
                  history, and culinary delights, where every corner reveals a new adventure."

money_essay = "I think a society without money could work, but it would be really hard.\
People would have to find a new way to trade things, like using food, skills, or services instead of money.\
For example, if someone wanted bread, they could trade vegetables or help the baker fix something.\
This sounds simple, but it could get really confusing if people don’t agree on what is fair."

example_essays = [health_org_essay, vacation_essay, money_essay]

generated_prompts = format.generate_prompts_for_texts(example_essays, format.GLOBAL_PIPE, batch_size=1)
print(generated_prompts)

['What is the main idea of this passage?', 'What is the best vacation for you?', 'What would happen if people didn’t have money?']


In [ ]:
# Our next idea is to add in-context learning.
# The idea in in-context learning is to enrich a prompt (= a prompt + essay couple) with additional tagged examples
# Instead of receiving a single example, the model will receive several semantically similar examples with their correct labels.
# The last example will be the actual test prompt, and the model will be asked to complete its label.
# How do we define semantically similar? Given a training example, run tf-idf on train dataset and find the five closest training examples.

In [ ]:
# This has proven to be successful for this task in a paper we relied on in this work:
# "OUTFOX: LLM-Generated Essay Detection Through In-Context Learning with Adversarially Generated Example" (Koike et al., 2023)
# In OUTFOX they added in-context learning and it increased robustness against adversarially crafted examples significantly.